# Chapter 6: Always Blocks and Processes

## Introduction

Always blocks are fundamental constructs in SystemVerilog that describe how hardware behaves over time. They define processes that execute continuously during simulation and represent different types of hardware structures. SystemVerilog provides three specialized always blocks that make design intent clearer and help avoid common modeling mistakes.

## Types of Always Blocks

SystemVerilog introduces three specialized always blocks:

- `always_comb` - For combinational logic
- `always_ff` - For sequential logic (flip-flops)
- `always_latch` - For latches

These replace the generic `always` block from Verilog and provide better checking and clearer intent.

### always_comb for Combinational Logic

The `always_comb` block is used to model combinational logic where outputs change immediately when inputs change.

#### Basic Syntax

```systemverilog
always_comb begin
    // Combinational logic statements
end
```

#### Key Features

- Automatically sensitive to all inputs (no sensitivity list needed)
- Executes immediately when any input changes
- Should not contain clocked logic or memory elements
- Helps catch incomplete sensitivity lists

#### Examples

##### Simple Multiplexer
```systemverilog
module mux2to1 (
    input  logic sel,
    input  logic a, b,
    output logic y
);

always_comb begin
    if (sel)
        y = b;
    else
        y = a;
end

endmodule
```

##### ALU Example
```systemverilog
module simple_alu (
    input  logic [3:0] a, b,
    input  logic [1:0] op,
    output logic [3:0] result,
    output logic       zero
);

always_comb begin
    case (op)
        2'b00: result = a + b;    // Add
        2'b01: result = a - b;    // Subtract
        2'b10: result = a & b;    // AND
        2'b11: result = a | b;    // OR
        default: result = 4'b0;
    endcase
    
    zero = (result == 4'b0);
end

endmodule
```

##### Priority Encoder
```systemverilog
module priority_encoder (
    input  logic [7:0] req,
    output logic [2:0] grant,
    output logic       valid
);

always_comb begin
    valid = |req;  // OR reduction - true if any bit set
    
    if (req[7])      grant = 3'd7;
    else if (req[6]) grant = 3'd6;
    else if (req[5]) grant = 3'd5;
    else if (req[4]) grant = 3'd4;
    else if (req[3]) grant = 3'd3;
    else if (req[2]) grant = 3'd2;
    else if (req[1]) grant = 3'd1;
    else if (req[0]) grant = 3'd0;
    else             grant = 3'd0;
end

endmodule
```

### always_ff for Sequential Logic

The `always_ff` block is used to model sequential logic elements like flip-flops and registers.

#### Basic Syntax

```systemverilog
always_ff @(posedge clk) begin
    // Sequential logic statements
end

// With reset
always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        // Reset logic
    end else begin
        // Normal clocked logic
    end
end
```

#### Key Features

- Must have a clocking event in sensitivity list
- Models memory elements (flip-flops, registers)
- Can include asynchronous reset/set
- Should use non-blocking assignments (<=) for clocked logic

#### Examples

##### Simple D Flip-Flop
```systemverilog
module dff (
    input  logic clk,
    input  logic rst_n,
    input  logic d,
    output logic q
);

always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n)
        q <= 1'b0;
    else
        q <= d;
end

endmodule
```

##### Counter with Enable
```systemverilog
module counter (
    input  logic       clk,
    input  logic       rst_n,
    input  logic       enable,
    input  logic       load,
    input  logic [7:0] load_value,
    output logic [7:0] count
);

always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        count <= 8'b0;
    end else if (load) begin
        count <= load_value;
    end else if (enable) begin
        count <= count + 1;
    end
    // If neither load nor enable, count maintains its value
end

endmodule
```

##### Shift Register
```systemverilog
module shift_register #(
    parameter WIDTH = 8
)(
    input  logic             clk,
    input  logic             rst_n,
    input  logic             shift_en,
    input  logic             serial_in,
    output logic             serial_out,
    output logic [WIDTH-1:0] parallel_out
);

logic [WIDTH-1:0] shift_reg;

always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        shift_reg <= '0;
    end else if (shift_en) begin
        shift_reg <= {shift_reg[WIDTH-2:0], serial_in};
    end
end

assign serial_out = shift_reg[WIDTH-1];
assign parallel_out = shift_reg;

endmodule
```

##### State Machine Example
```systemverilog
typedef enum logic [1:0] {
    IDLE = 2'b00,
    ACTIVE = 2'b01,
    WAIT = 2'b10,
    DONE = 2'b11
} state_t;

module fsm (
    input  logic   clk,
    input  logic   rst_n,
    input  logic   start,
    input  logic   ready,
    output logic   busy,
    output logic   complete
);

state_t current_state, next_state;

// State register
always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n)
        current_state <= IDLE;
    else
        current_state <= next_state;
end

// Next state logic (combinational)
always_comb begin
    case (current_state)
        IDLE: begin
            if (start)
                next_state = ACTIVE;
            else
                next_state = IDLE;
        end
        
        ACTIVE: begin
            if (ready)
                next_state = WAIT;
            else
                next_state = ACTIVE;
        end
        
        WAIT: begin
            next_state = DONE;
        end
        
        DONE: begin
            next_state = IDLE;
        end
        
        default: next_state = IDLE;
    endcase
end

// Output logic (combinational)
always_comb begin
    busy = (current_state != IDLE) && (current_state != DONE);
    complete = (current_state == DONE);
end

endmodule
```

### always_latch for Latches

The `always_latch` block is used to model transparent latches, though latches are generally discouraged in synchronous design.

#### Basic Syntax

```systemverilog
always_latch begin
    // Latch logic statements
end
```

#### Key Features

- Models level-sensitive storage elements
- Should be avoided in most synchronous designs
- Can cause timing issues and make verification difficult
- Sometimes used for specific analog or mixed-signal applications

#### Example

```systemverilog
module d_latch (
    input  logic enable,
    input  logic d,
    output logic q
);

always_latch begin
    if (enable)
        q = d;
    // When enable is low, q retains its value (latch behavior)
end

endmodule
```

### Blocking vs. Non-Blocking Assignments

Understanding the difference between blocking (=) and non-blocking (<=) assignments is crucial for proper hardware modeling.

#### Blocking Assignments (=)

- Execute immediately in sequence
- Used in combinational logic (`always_comb`)
- Model wire-like behavior
- Can create race conditions if misused

#### Non-Blocking Assignments (<=)

- Scheduled to execute at end of time step
- Used in sequential logic (`always_ff`)
- Model register-like behavior
- Prevent race conditions in clocked logic

#### Examples Comparing Both

##### Combinational Logic - Use Blocking (=)
```systemverilog
// Correct - using blocking assignments
always_comb begin
    temp = a & b;
    y = temp | c;
end

// Incorrect - non-blocking in combinational logic
always_comb begin
    temp <= a & b;  // Wrong!
    y <= temp | c;  // Wrong! - temp not updated yet
end
```

##### Sequential Logic - Use Non-Blocking (<=)
```systemverilog
// Correct - using non-blocking assignments
always_ff @(posedge clk) begin
    q1 <= d;
    q2 <= q1;  // Creates shift register
end

// Incorrect - blocking in sequential logic
always_ff @(posedge clk) begin
    q1 = d;
    q2 = q1;   // Both update to 'd' simultaneously - not a shift register!
end
```

### Best Practices Summary

1. **Combinational logic (`always_comb`)**: Use blocking assignments (=)
2. **Sequential logic (`always_ff`)**: Use non-blocking assignments (<=)
3. **Mixed assignments**: Never mix blocking and non-blocking in the same always block

### Race Conditions and Common Pitfalls

#### Race Condition Example

```systemverilog
// Problematic code - race condition
module race_example (
    input  logic clk,
    input  logic d,
    output logic q1, q2
);

// Two separate always blocks updating at same time
always_ff @(posedge clk) begin
    q1 <= d;
end

always_ff @(posedge clk) begin
    q2 <= q1;  // Race condition! Order of execution matters
end

endmodule
```

```systemverilog
// Better approach - combine into one always block
always_ff @(posedge clk) begin
    q1 <= d;
    q2 <= q1;  // Now guaranteed to work correctly
end
```

#### Incomplete Sensitivity Lists (Verilog issue fixed by always_comb)

```systemverilog
// Old Verilog style - error prone
always @(a, b) begin  // Forgot 'c' in sensitivity list!
    y = a & b;
    z = y | c;        // 'z' won't update when 'c' changes
end

// SystemVerilog solution - automatic sensitivity
always_comb begin
    y = a & b;
    z = y | c;        // Automatically sensitive to a, b, and c
end
```

#### Process Control and Advanced Topics

#### Multiple Clock Domains

```systemverilog
module dual_clock_design (
    input  logic clk1, clk2,
    input  logic rst_n,
    input  logic data_in,
    output logic data_out
);

logic ff1, ff2;

// Clock domain 1
always_ff @(posedge clk1 or negedge rst_n) begin
    if (!rst_n)
        ff1 <= 1'b0;
    else
        ff1 <= data_in;
end

// Clock domain 2
always_ff @(posedge clk2 or negedge rst_n) begin
    if (!rst_n)
        ff2 <= 1'b0;
    else
        ff2 <= ff1;  // Potential metastability issue!
end

assign data_out = ff2;

endmodule
```

#### Generate Blocks with Always Blocks

```systemverilog
module parameterized_register #(
    parameter WIDTH = 8,
    parameter STAGES = 4
)(
    input  logic             clk,
    input  logic             rst_n,
    input  logic [WIDTH-1:0] data_in,
    output logic [WIDTH-1:0] data_out
);

logic [WIDTH-1:0] pipe_reg [STAGES-1:0];

genvar i;
generate
    for (i = 0; i < STAGES; i++) begin : pipe_stage
        always_ff @(posedge clk or negedge rst_n) begin
            if (!rst_n) begin
                pipe_reg[i] <= '0;
            end else begin
                if (i == 0)
                    pipe_reg[i] <= data_in;
                else
                    pipe_reg[i] <= pipe_reg[i-1];
            end
        end
    end
endgenerate

assign data_out = pipe_reg[STAGES-1];

endmodule
```

### Best Practices and Guidelines

#### Design Guidelines

1. **Use appropriate always block types**:
   - `always_comb` for combinational logic
   - `always_ff` for sequential logic
   - Avoid `always_latch` unless specifically needed

2. **Assignment types**:
   - Blocking (=) in `always_comb`
   - Non-blocking (<=) in `always_ff`
   - Never mix both types in the same block

3. **Reset strategy**:
   - Use asynchronous reset for critical paths
   - Consider synchronous reset for better timing
   - Initialize all registers consistently

4. **Avoid common mistakes**:
   - Incomplete case statements
   - Inferred latches from incomplete if-else chains
   - Multiple drivers to same signal

#### Code Style Example

```systemverilog
module good_style_example (
    input  logic       clk,
    input  logic       rst_n,
    input  logic [7:0] data_in,
    input  logic       enable,
    output logic [7:0] data_out,
    output logic       valid
);

// Sequential logic - use always_ff with non-blocking
always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        data_out <= 8'b0;
        valid    <= 1'b0;
    end else if (enable) begin
        data_out <= data_in;
        valid    <= 1'b1;
    end else begin
        valid    <= 1'b0;
        // data_out retains its value
    end
end

endmodule
```

### Summary

Always blocks are the cornerstone of behavioral modeling in SystemVerilog. The three specialized types (`always_comb`, `always_ff`, `always_latch`) provide clear intent and help prevent common coding errors. Understanding when to use blocking vs. non-blocking assignments is crucial for creating hardware that behaves as intended. Following the best practices outlined in this chapter will lead to more reliable, synthesizable, and maintainable code.


### Key Takeaways

- Use `always_comb` for combinational logic with blocking assignments (=)
- Use `always_ff` for sequential logic with non-blocking assignments (<=)
- Avoid `always_latch` unless specifically required
- Be consistent with reset strategies
- Understand race conditions and how to avoid them
- Always consider the hardware implications of your code